# Parte 2


El propósito de este ejercicio es construir desde cero un modelo híbrido que combine los tres paradigmas (MBA, DS y DES) utilizando Python y la biblioteca SimPy. El objetivo es simular una población de agentes cuya dinámica interna (DS) los lleva a requerir un servicio con recursos limitados (DES), todo dentro de un entorno deinteracciones espaciales (MBA).
Usted modelará una población de agentes que se mueven en un espacio.

* Componente DS: Cada agente tiene un nivel interno de "Energía" que funciona como un stock. Este se recupera lentamente con el tiempo (inflow) pero se gasta con el movimiento y las interacciones (outflow).

* Componente MBA: Los agentes se mueven e interactúan con otros agentes cercanos. Estas interacciones pueden ser positivas (ganan energía) o negativas (pierden energía).

* Componente DES: Si la energía de un agente cae por debajo de un umbral crítico, debe buscar una "Estación de Recarga". Esta estación es un recurso limitado y compartido, por lo que los agentes pueden tener que esperar en una cola para usarla

## Ejercicio

### Importar Bibliotecas

In [12]:
import simpy
import random
import numpy as np

### Definir Parámetros

In [13]:
# Parámetros de la simulación
NUM_AGENTES = 20
ANCHO_MUNDO = 100
ALTO_MUNDO = 100

TASA_RECUPERACION_NATURAL = 0.05
GASTO_POR_MOVIMIENTO = 0.15
EFECTO_INTERACCION = 0.5

NUM_PUESTOS_RECARGA = 5
ENERGIA_CRITICA = 2.0
TIEMPO_RECARGA = 10.0

TIEMPO_SIMULACION = 100
DT = 1.0


In [14]:
# Distancia para detectar interacciones entre agentes
DISTANCIA_INTERACCION = 15.0

# Listas para recolectar datos
historial_energia_promedio = []
historial_agentes_en_cola = []
historial_tiempo = []

### Crear la Clase / Atributos MBA y DS: Dentro del constructor, inicialice los atributos del agente / Proceso de Recarga (DES) /Método de Actualización

In [15]:
class Agente:
    def __init__(self, id, posicion_inicial, env, estacion_recarga):
        self.id = id
        self.posicion = np.array(posicion_inicial, dtype=float)
        self.velocidad = np.array([random.uniform(-1, 1), random.uniform(-1, 1)])
        self.env = env
        self.estacion_recarga = estacion_recarga
        self.energia = 10.0
        self.en_recarga = False  # Flag para evitar múltiples solicitudes
        
    def proceso_recarga(self):
        """Proceso DES de recarga de energía"""
        self.en_recarga = True
        print(f'  [{self.env.now:.1f}] Agente {self.id} solicita recarga (energía={self.energia:.2f})')
        
        with self.estacion_recarga.request() as req:
            yield req
            print(f'  [{self.env.now:.1f}] Agente {self.id} inicia recarga')
            yield self.env.timeout(TIEMPO_RECARGA)
            self.energia = 10.0
            print(f'  [{self.env.now:.1f}] Agente {self.id} completa recarga')
        
        self.en_recarga = False
    
    def actualizar_estado(self, efecto_interaccion_neto):
        """
        Actualiza el estado del agente integrando DS, MBA y DES
        
        Args:
            efecto_interaccion_neto: Cambio de energía por interacciones con otros agentes
        """
        # === COMPONENTE DS: Actualizar energía ===
        inflow = TASA_RECUPERACION_NATURAL * DT
        outflow = GASTO_POR_MOVIMIENTO * DT
        self.energia += inflow - outflow + efecto_interaccion_neto
        
        # Limitar energía entre 0 y 10
        self.energia = max(0.0, min(10.0, self.energia))
        
        # === COMPONENTE MBA: Actualizar posición ===
        self.posicion += self.velocidad * DT
        
        # Rebotar en los bordes
        if self.posicion[0] <= 0 or self.posicion[0] >= ANCHO_MUNDO:
            self.velocidad[0] *= -1
            self.posicion[0] = np.clip(self.posicion[0], 0, ANCHO_MUNDO)
        if self.posicion[1] <= 0 or self.posicion[1] >= ALTO_MUNDO:
            self.velocidad[1] *= -1
            self.posicion[1] = np.clip(self.posicion[1], 0, ALTO_MUNDO)
        
        # === PUENTE MBA → DES: Verificar necesidad de recarga ===
        if self.energia < ENERGIA_CRITICA and not self.en_recarga:
            self.env.process(self.proceso_recarga())


### Función de Simulación Principal / Bucle de Simulación


In [16]:
def ejecutar_simulacion(env, agentes, TIEMPO_SIMULACION):
    """
    Función generadora principal que ejecuta el bucle de simulación
    
    Args:
        env: Entorno de SimPy
        agentes: Lista de agentes
        TIEMPO_SIMULACION: Duración total de la simulación
    """
    print(f"\nIniciando simulación con {len(agentes)} agentes\n")
    
    # Bucle principal de simulación
    while env.now < TIEMPO_SIMULACION:
        
        # === INTERACCIONES MBA ===
        # Calcular efecto neto de interacciones para cada agente
        efecto_interaccion_neto = {agente.id: 0.0 for agente in agentes}
        
        # Revisar todos los pares de agentes
        for i in range(len(agentes)):
            for j in range(i + 1, len(agentes)):
                agente_i = agentes[i]
                agente_j = agentes[j]
                
                # Calcular distancia euclidiana entre agentes
                distancia = np.linalg.norm(agente_i.posicion - agente_j.posicion)
                
                # Si están suficientemente cerca, interactúan
                if distancia < DISTANCIA_INTERACCION:
                    # Interacción aleatoria: puede ser positiva o negativa
                    efecto = random.choice([-1, 1]) * EFECTO_INTERACCION
                    
                    # Aplicar efecto a ambos agentes
                    efecto_interaccion_neto[agente_i.id] += efecto
                    efecto_interaccion_neto[agente_j.id] += efecto
        
        # === ACTUALIZACIÓN DE AGENTES ===
        for agente in agentes:
            agente.actualizar_estado(efecto_interaccion_neto[agente.id])
        
        # === RECOLECCIÓN DE DATOS ===
        energia_promedio = sum(a.energia for a in agentes) / len(agentes)
        num_agentes_en_cola = sum(1 for a in agentes if a.en_recarga)
        
        historial_energia_promedio.append(energia_promedio)
        historial_agentes_en_cola.append(num_agentes_en_cola)
        historial_tiempo.append(env.now)
        
        # === AVANCE DEL TIEMPO ===
        yield env.timeout(DT)
    
    print(f"\nSimulación completada en t={env.now}\n")

In [17]:
def main():
    """Función principal que configura y ejecuta todo"""
    
    print("MODELO HÍBRIDO: MBA + DS + DES")
    print(f"Parámetros:")
    print(f"  Agentes: {NUM_AGENTES}")
    print(f"  Mundo: {ANCHO_MUNDO}x{ALTO_MUNDO}")
    print(f"  Puestos de recarga: {NUM_PUESTOS_RECARGA}")
    print(f"  Energía crítica: {ENERGIA_CRITICA}")
    print(f"  Tiempo de simulación: {TIEMPO_SIMULACION}")
    
    # Limpiar historial (por si se ejecuta múltiples veces)
    global historial_energia_promedio, historial_agentes_en_cola, historial_tiempo
    historial_energia_promedio = []
    historial_agentes_en_cola = []
    historial_tiempo = []
    
    # Crear el entorno de SimPy
    env = simpy.Environment()
    
    # Crear el recurso de estación de recarga
    estacion_recarga = simpy.Resource(env, capacity=NUM_PUESTOS_RECARGA)
    
    # Crear la lista de agentes con posiciones iniciales aleatorias
    agentes = []
    for i in range(NUM_AGENTES):
        posicion_inicial = [
            random.uniform(0, ANCHO_MUNDO),
            random.uniform(0, ALTO_MUNDO)
        ]
        agente = Agente(i, posicion_inicial, env, estacion_recarga)
        agentes.append(agente)
    
    # Iniciar el proceso de simulación
    env.process(ejecutar_simulacion(env, agentes, TIEMPO_SIMULACION))
    
    # Ejecutar la simulación
    env.run()
    
    return agentes

### Análisis Final

In [18]:
def analizar_resultados(agentes):
    """Análisis final y visualización de estadísticas"""
    
    print("ANÁLISIS FINAL DE RESULTADOS")
    
    # Estadísticas de energía a lo largo del tiempo
    if historial_energia_promedio:
        print("\nESTADÍSTICAS DE ENERGÍA:")
        print(f"  Energía promedio final: {historial_energia_promedio[-1]:.2f}")
        print(f"  Energía promedio global: {np.mean(historial_energia_promedio):.2f}")
        print(f"  Energía máxima observada: {max(historial_energia_promedio):.2f}")
        print(f"  Energía mínima observada: {min(historial_energia_promedio):.2f}")
        print(f"  Desviación estándar: {np.std(historial_energia_promedio):.2f}")
    
    # Estadísticas de uso de estación de recarga
    if historial_agentes_en_cola:
        print("\nESTADÍSTICAS DE ESTACIÓN DE RECARGA:")
        print(f"  Promedio de agentes en recarga: {np.mean(historial_agentes_en_cola):.2f}")
        print(f"  Máximo simultáneo en recarga: {max(historial_agentes_en_cola)}")
        
        # Calcular tiempo con estación saturada
        pasos_saturados = sum(1 for n in historial_agentes_en_cola if n >= NUM_PUESTOS_RECARGA)
        porcentaje_saturacion = (pasos_saturados / len(historial_agentes_en_cola)) * 100
        print(f"  % tiempo con estación saturada: {porcentaje_saturacion:.1f}%")
    
    # Estado final de los agentes
    energias_finales = [a.energia for a in agentes]
    print("\nESTADO FINAL DE LOS AGENTES:")
    print(f"  Energía promedio: {np.mean(energias_finales):.2f}")
    print(f"  Energía máxima: {max(energias_finales):.2f}")
    print(f"  Energía mínima: {min(energias_finales):.2f}")
    print(f"  Agentes con energía crítica: {sum(1 for e in energias_finales if e < ENERGIA_CRITICA)}")
    print(f"  Agentes con energía alta (>8): {sum(1 for e in energias_finales if e > 8)}")
    
    # Distribución de posiciones finales
    print("\nDISTRIBUCIÓN ESPACIAL FINAL:")
    pos_x = [a.posicion[0] for a in agentes]
    pos_y = [a.posicion[1] for a in agentes]
    print(f"  Centro de masa: ({np.mean(pos_x):.1f}, {np.mean(pos_y):.1f})")
    print(f"  Dispersión X: {np.std(pos_x):.1f}")
    print(f"  Dispersión Y: {np.std(pos_y):.1f}")
    


In [19]:
# Bloque principal de ejecución
if __name__ == "__main__":
    # Establecer semilla para reproducibilidad
    random.seed(42)
    np.random.seed(42)
    
    # Ejecutar simulación
    agentes = main()
    
    # Analizar resultados
    analizar_resultados(agentes)

MODELO HÍBRIDO: MBA + DS + DES
Parámetros:
  Agentes: 20
  Mundo: 100x100
  Puestos de recarga: 5
  Energía crítica: 2.0
  Tiempo de simulación: 100

Iniciando simulación con 20 agentes

  [17.0] Agente 5 solicita recarga (energía=1.50)
  [17.0] Agente 5 inicia recarga
  [19.0] Agente 1 solicita recarga (energía=1.60)
  [19.0] Agente 1 inicia recarga
  [27.0] Agente 5 completa recarga
  [29.0] Agente 1 completa recarga
  [33.0] Agente 16 solicita recarga (energía=1.40)
  [33.0] Agente 16 inicia recarga
  [35.0] Agente 6 solicita recarga (energía=1.90)
  [35.0] Agente 6 inicia recarga
  [42.0] Agente 0 solicita recarga (energía=1.40)
  [42.0] Agente 0 inicia recarga
  [43.0] Agente 16 completa recarga
  [45.0] Agente 6 completa recarga
  [47.0] Agente 11 solicita recarga (energía=1.70)
  [47.0] Agente 11 inicia recarga
  [52.0] Agente 0 completa recarga
  [54.0] Agente 5 solicita recarga (energía=1.20)
  [54.0] Agente 5 inicia recarga
  [55.0] Agente 3 solicita recarga (energía=1.90)
  

## PREGUNTA 1: Capacidad Generosa (NUM_PUESTOS_RECARGA = 5)

In [ ]:
# Configurar parámetros
NUM_PUESTOS_RECARGA = 5
TIEMPO_SIMULACION = 200 

In [21]:
# Establecer semilla para reproducibilidad
random.seed(42)
np.random.seed(42)

# Ejecutar simulación
agentes = main()

# Analizar resultados
analizar_resultados(agentes)


MODELO HÍBRIDO: MBA + DS + DES
Parámetros:
  Agentes: 20
  Mundo: 100x100
  Puestos de recarga: 5
  Energía crítica: 2.0
  Tiempo de simulación: 200

Iniciando simulación con 20 agentes

  [17.0] Agente 5 solicita recarga (energía=1.50)
  [17.0] Agente 5 inicia recarga
  [19.0] Agente 1 solicita recarga (energía=1.60)
  [19.0] Agente 1 inicia recarga
  [27.0] Agente 5 completa recarga
  [29.0] Agente 1 completa recarga
  [33.0] Agente 16 solicita recarga (energía=1.40)
  [33.0] Agente 16 inicia recarga
  [35.0] Agente 6 solicita recarga (energía=1.90)
  [35.0] Agente 6 inicia recarga
  [42.0] Agente 0 solicita recarga (energía=1.40)
  [42.0] Agente 0 inicia recarga
  [43.0] Agente 16 completa recarga
  [45.0] Agente 6 completa recarga
  [47.0] Agente 11 solicita recarga (energía=1.70)
  [47.0] Agente 11 inicia recarga
  [52.0] Agente 0 completa recarga
  [54.0] Agente 5 solicita recarga (energía=1.20)
  [54.0] Agente 5 inicia recarga
  [55.0] Agente 3 solicita recarga (energía=1.90)
  

### Cuántos agentes, en promedio, utilizan la estación durante la simulación?

In [24]:
if historial_agentes_en_cola:
    print(f"Promedio de agentes en proceso de recarga: {np.mean(historial_agentes_en_cola):.2f}")
    print(f"Máximo de agentes simultáneos en recarga: {max(historial_agentes_en_cola)}")
    

Promedio de agentes en proceso de recarga: 1.66
Máximo de agentes simultáneos en recarga: 5


### ¿Se forman colas significativas o el servicio es fluido?

In [23]:
if historial_agentes_en_cola:
    pasos_con_espera = sum(1 for n in historial_agentes_en_cola if n >= NUM_PUESTOS_RECARGA)
    porcentaje_saturacion = (pasos_con_espera / len(historial_agentes_en_cola)) * 100
    print(f"La estación estuvo saturada el {porcentaje_saturacion:.1f}% del tiempo")
    
    if porcentaje_saturacion < 5:
        print("El servicio es FLUIDO. Las colas son prácticamente inexistentes.")
    elif porcentaje_saturacion < 20:
        print("El servicio es MAYORMENTE FLUIDO con colas ocasionales.")
    else:
        print("Se forman colas SIGNIFICATIVAS.")
    
    print(f"\nUtilización promedio: {(np.mean(historial_agentes_en_cola)/NUM_PUESTOS_RECARGA)*100:.1f}%")

La estación estuvo saturada el 0.5% del tiempo
El servicio es FLUIDO. Las colas son prácticamente inexistentes.

Utilización promedio: 33.1%


## PREGUNTA 2: Cuello de Botella (NUM_PUESTOS_RECARGA = 1)

In [25]:
# Configurar parámetros
NUM_PUESTOS_RECARGA = 1  # Capacidad drásticamente reducida
TIEMPO_SIMULACION = 200

In [33]:
# Establecer semilla para reproducibilidad
random.seed(42)
np.random.seed(42)


# Ejecutar simulación
agentes = main()

# Analizar resultados
analizar_resultados(agentes)

MODELO HÍBRIDO: MBA + DS + DES
Parámetros:
  Agentes: 20
  Mundo: 100x100
  Puestos de recarga: 1
  Energía crítica: 2.0
  Tiempo de simulación: 200

Iniciando simulación con 20 agentes

  [17.0] Agente 5 solicita recarga (energía=1.50)
  [17.0] Agente 5 inicia recarga
  [19.0] Agente 1 solicita recarga (energía=1.60)
  [27.0] Agente 5 completa recarga
  [27.0] Agente 1 inicia recarga
  [33.0] Agente 16 solicita recarga (energía=1.40)
  [35.0] Agente 6 solicita recarga (energía=1.90)
  [37.0] Agente 1 completa recarga
  [37.0] Agente 16 inicia recarga
  [42.0] Agente 0 solicita recarga (energía=1.40)
  [47.0] Agente 16 completa recarga
  [47.0] Agente 11 solicita recarga (energía=1.70)
  [47.0] Agente 6 inicia recarga
  [54.0] Agente 5 solicita recarga (energía=1.20)
  [55.0] Agente 3 solicita recarga (energía=1.90)
  [57.0] Agente 6 completa recarga
  [57.0] Agente 0 inicia recarga
  [60.0] Agente 18 solicita recarga (energía=1.40)
  [67.0] Agente 0 completa recarga
  [67.0] Agente 11

### ¿Cómo impacta este cambio en la dinámica general de energía?

In [34]:
print ("¿Cómo impacta este cambio en la dinámica general de energía?")
print(f"Energía promedio de la población: {np.mean(historial_energia_promedio):.2f}")
print(f"Energía mínima observada: {min(historial_energia_promedio):.2f}")
print(f"Desviación estándar de energía: {np.std(historial_energia_promedio):.2f}")

¿Cómo impacta este cambio en la dinámica general de energía?
Energía promedio de la población: 4.85
Energía mínima observada: 3.15
Desviación estándar de energía: 1.31


### Comparar con escenario anterior (valores de referencia con 5 puestos)


In [29]:
print("\n  Comparación con escenario de 5 puestos (Pregunta 1):")
print(f"Energía promedio: 6.24 (5 puestos) → {np.mean(historial_energia_promedio):.2f} (1 puesto)")
diferencia_energia = ((np.mean(historial_energia_promedio) / 6.24) - 1) * 100
print(f"Cambio: {diferencia_energia:+.1f}%")


  Comparación con escenario de 5 puestos (Pregunta 1):
Energía promedio: 6.24 (5 puestos) → 4.85 (1 puesto)
Cambio: -22.2%


### ¿Mayor número de agentes con energía crítica por períodos prolongados?

In [30]:
# Contar cuántos pasos de tiempo hay agentes en energía crítica
agentes_criticos_por_paso = []
for t_idx in range(len(historial_tiempo)):
    # Aquí necesitaríamos el historial individual, pero podemos inferir
    pass

In [36]:
# Analizar estado final
energias_finales = [a.energia for a in agentes]
agentes_criticos_final = sum(1 for e in energias_finales if e < ENERGIA_CRITICA)
agentes_energia_baja = sum(1 for e in energias_finales if e < 4.0)

print("¿Mayor número de agentes con energía crítica por períodos prolongados?")
print(f"Agentes con energía crítica al final: {agentes_criticos_final} de {len(agentes)}")
print(f"Agentes con energía baja (<4.0) al final: {agentes_energia_baja} de {len(agentes)}")
print(f"Energía promedio al final: {np.mean(energias_finales):.2f}")
print(f"Rango de energías finales: [{min(energias_finales):.2f}, {max(energias_finales):.2f}]")

if historial_agentes_en_cola:
    cola_promedio = np.mean(historial_agentes_en_cola)
    saturacion = (sum(1 for n in historial_agentes_en_cola if n >= NUM_PUESTOS_RECARGA) / len(historial_agentes_en_cola)) * 100
    
    print(f"La estación estuvo saturada el {saturacion:.1f}% del tiempo")
    print(f"Promedio de agentes esperando/en recarga: {cola_promedio:.2f}")
    print(f"Máximo de agentes en cola simultáneos: {max(historial_agentes_en_cola)}")

¿Mayor número de agentes con energía crítica por períodos prolongados?
Agentes con energía crítica al final: 0 de 20
Agentes con energía baja (<4.0) al final: 2 de 20
Energía promedio al final: 8.15
Rango de energías finales: [2.00, 10.00]
La estación estuvo saturada el 91.0% del tiempo
Promedio de agentes esperando/en recarga: 6.92
Máximo de agentes en cola simultáneos: 11


## PREGUNTA 3: Efecto del Cuello de Botella sobre Comportamiento Agregado


In [43]:
# Configurar parámetros
NUM_PUESTOS_RECARGA = 1  # Escenario con cuello de botella
TIEMPO_SIMULACION = 200

# Establecer semilla para reproducibilidad
random.seed(42)
np.random.seed(42)


In [44]:
# Ejecutar simulación
agentes = main()

# Analizar resultados
analizar_resultados(agentes)


MODELO HÍBRIDO: MBA + DS + DES
Parámetros:
  Agentes: 20
  Mundo: 100x100
  Puestos de recarga: 1
  Energía crítica: 2.0
  Tiempo de simulación: 200

Iniciando simulación con 20 agentes

  [17.0] Agente 5 solicita recarga (energía=1.50)
  [17.0] Agente 5 inicia recarga
  [19.0] Agente 1 solicita recarga (energía=1.60)
  [27.0] Agente 5 completa recarga
  [27.0] Agente 1 inicia recarga
  [33.0] Agente 16 solicita recarga (energía=1.40)
  [35.0] Agente 6 solicita recarga (energía=1.90)
  [37.0] Agente 1 completa recarga
  [37.0] Agente 16 inicia recarga
  [42.0] Agente 0 solicita recarga (energía=1.40)
  [47.0] Agente 16 completa recarga
  [47.0] Agente 11 solicita recarga (energía=1.70)
  [47.0] Agente 6 inicia recarga
  [54.0] Agente 5 solicita recarga (energía=1.20)
  [55.0] Agente 3 solicita recarga (energía=1.90)
  [57.0] Agente 6 completa recarga
  [57.0] Agente 0 inicia recarga
  [60.0] Agente 18 solicita recarga (energía=1.40)
  [67.0] Agente 0 completa recarga
  [67.0] Agente 11

### ¿Qué efecto observable tiene el cuello de botella DES sobre el comportamiento agregado del sistema (MBA+DS)?

In [45]:
print("¿Qué efecto observable tiene el cuello de botella DES sobre el comportamiento agregado del sistema (MBA+DS)?")

if historial_energia_promedio and historial_tiempo:
    # Análisis de la evolución temporal de la energía    
    # Dividir la simulación en cuartiles
    n_pasos = len(historial_energia_promedio)
    cuartil1 = historial_energia_promedio[:n_pasos//4]
    cuartil2 = historial_energia_promedio[n_pasos//4:n_pasos//2]
    cuartil3 = historial_energia_promedio[n_pasos//2:3*n_pasos//4]
    cuartil4 = historial_energia_promedio[3*n_pasos//4:]
    
    print(f"Energía promedio Q1 (t=0-50):    {np.mean(cuartil1):.2f}")
    print(f"Energía promedio Q2 (t=50-100):  {np.mean(cuartil2):.2f}")
    print(f"Energía promedio Q3 (t=100-150): {np.mean(cuartil3):.2f}")
    print(f"Energía promedio Q4 (t=150-200): {np.mean(cuartil4):.2f}")
    
    # Tendencia
    energia_inicial = np.mean(historial_energia_promedio[:20])
    energia_final = np.mean(historial_energia_promedio[-20])
    cambio_total = energia_final - energia_inicial
    print(f"\n Energía al inicio (primeros 20 pasos): {energia_inicial:.2f}")
    print(f"Energía al final (últimos 20 pasos):   {energia_final:.2f}")
    print(f" total: {cambio_total:+.2f} ({(cambio_total/energia_inicial)*100:+.1f}%)")
    
    if cambio_total < -0.5:
        print("  → TENDENCIA DESCENDENTE: El sistema sufre deterioro energético progresivo")
    elif cambio_total > 0.5:
        print("  → TENDENCIA ASCENDENTE: El sistema se recupera con el tiempo")
    else:
        print("  → TENDENCIA ESTABLE: El sistema alcanza un equilibrio dinámico")


¿Qué efecto observable tiene el cuello de botella DES sobre el comportamiento agregado del sistema (MBA+DS)?
Energía promedio Q1 (t=0-50):    6.64
Energía promedio Q2 (t=50-100):  4.90
Energía promedio Q3 (t=100-150): 4.15
Energía promedio Q4 (t=150-200): 3.72

 Energía al inicio (primeros 20 pasos): 7.66
Energía al final (últimos 20 pasos):   4.20
 total: -3.46 (-45.2%)
  → TENDENCIA DESCENDENTE: El sistema sufre deterioro energético progresivo


### Comparación con otro escenario sin cuello de botella

In [46]:
print("Comparación con otro escenario sin cuello de botella")
print("  Referencia con 5 puestos (Pregunta 1):")
print(f"  • Energía promedio: 6.24 (5 puestos) vs {np.mean(historial_energia_promedio):.2f} (1 puesto)")
print(f"  • Reducción: {((np.mean(historial_energia_promedio)/6.24 - 1)*100):.1f}%")
print(f"  • Variabilidad: 0.77 (5 puestos) vs {np.std(historial_energia_promedio):.2f} (1 puesto)")
print(f"  • Incremento en variabilidad: {((np.std(historial_energia_promedio)/0.77 - 1)*100):.1f}%")

Comparación con otro escenario sin cuello de botella
  Referencia con 5 puestos (Pregunta 1):
  • Energía promedio: 6.24 (5 puestos) vs 4.85 (1 puesto)
  • Reducción: -22.2%
  • Variabilidad: 0.77 (5 puestos) vs 1.31 (1 puesto)
  • Incremento en variabilidad: 70.5%


### Impacto del componente DES en el sistema agregado

In [47]:
if historial_agentes_en_cola:
    print("Impacto del componente DES en el sistema agregado")
    saturacion = (sum(1 for n in historial_agentes_en_cola if n >= NUM_PUESTOS_RECARGA) / len(historial_agentes_en_cola)) * 100
    print(f"Saturación de la estación: {saturacion:.1f}% del tiempo")
    print(f"Promedio de agentes en cola/recarga: {np.mean(historial_agentes_en_cola):.2f}")
    print(f"Máximo de agentes en cola: {max(historial_agentes_en_cola)}")

Impacto del componente DES en el sistema agregado
Saturación de la estación: 91.0% del tiempo
Promedio de agentes en cola/recarga: 6.92
Máximo de agentes en cola: 11


## PREGUNTA 4: Bucle de Retroalimentación y Comportamiento Emergente

In [49]:
# Función para ejecutar un experimento y recolectar resultados
def ejecutar_experimento(num_puestos, mostrar_log=False):
    """Ejecuta una simulación completa y retorna métricas"""
        # Parámetros
    NUM_AGENTES = 20
    ANCHO_MUNDO = 100
    ALTO_MUNDO = 100
    TASA_RECUPERACION_NATURAL = 0.05
    GASTO_POR_MOVIMIENTO = 0.15
    EFECTO_INTERACCION = 0.5
    ENERGIA_CRITICA = 2.0
    TIEMPO_RECARGA = 10.0
    TIEMPO_SIMULACION = 200
    DT = 1.0
    DISTANCIA_INTERACCION = 15.0
    
    historial_energia_promedio = []
    historial_agentes_en_cola = []
    historial_tiempo = []
    
    class Agente:
        def __init__(self, id, posicion_inicial, env, estacion_recarga):
            self.id = id
            self.posicion = np.array(posicion_inicial, dtype=float)
            self.velocidad = np.array([random.uniform(-1, 1), random.uniform(-1, 1)])
            self.env = env
            self.estacion_recarga = estacion_recarga
            self.energia = 10.0
            self.en_recarga = False
            self.num_recargas = 0
            
        def proceso_recarga(self):
            self.en_recarga = True
            self.num_recargas += 1
            with self.estacion_recarga.request() as req:
                yield req
                yield self.env.timeout(TIEMPO_RECARGA)
                self.energia = 10.0
            self.en_recarga = False
        
        def actualizar_estado(self, efecto_interaccion_neto):
            inflow = TASA_RECUPERACION_NATURAL * DT
            outflow = GASTO_POR_MOVIMIENTO * DT
            self.energia += inflow - outflow + efecto_interaccion_neto
            self.energia = max(0.0, min(10.0, self.energia))
            
            self.posicion += self.velocidad * DT
            if self.posicion[0] <= 0 or self.posicion[0] >= ANCHO_MUNDO:
                self.velocidad[0] *= -1
                self.posicion[0] = np.clip(self.posicion[0], 0, ANCHO_MUNDO)
            if self.posicion[1] <= 0 or self.posicion[1] >= ALTO_MUNDO:
                self.velocidad[1] *= -1
                self.posicion[1] = np.clip(self.posicion[1], 0, ALTO_MUNDO)
            
            if self.energia < ENERGIA_CRITICA and not self.en_recarga:
                self.env.process(self.proceso_recarga())
    
    def ejecutar_simulacion(env, agentes, tiempo_simulacion):
        while env.now < tiempo_simulacion:
            efecto_interaccion_neto = {agente.id: 0.0 for agente in agentes}
            
            for i in range(len(agentes)):
                for j in range(i + 1, len(agentes)):
                    agente_i = agentes[i]
                    agente_j = agentes[j]
                    distancia = np.linalg.norm(agente_i.posicion - agente_j.posicion)
                    if distancia < DISTANCIA_INTERACCION:
                        efecto = random.choice([-1, 1]) * EFECTO_INTERACCION
                        efecto_interaccion_neto[agente_i.id] += efecto
                        efecto_interaccion_neto[agente_j.id] += efecto
            
            for agente in agentes:
                agente.actualizar_estado(efecto_interaccion_neto[agente.id])
            
            energia_promedio = sum(a.energia for a in agentes) / len(agentes)
            num_agentes_en_cola = sum(1 for a in agentes if a.en_recarga)
            
            historial_energia_promedio.append(energia_promedio)
            historial_agentes_en_cola.append(num_agentes_en_cola)
            historial_tiempo.append(env.now)
            
            yield env.timeout(DT)
    
    # Ejecutar
    random.seed(42)
    np.random.seed(42)
    env = simpy.Environment()
    estacion_recarga = simpy.Resource(env, capacity=num_puestos)
    agentes = [Agente(i, [random.uniform(0, ANCHO_MUNDO), random.uniform(0, ALTO_MUNDO)], 
                      env, estacion_recarga) for i in range(NUM_AGENTES)]
    env.process(ejecutar_simulacion(env, agentes, TIEMPO_SIMULACION))
    env.run()
    
    return {
        'num_puestos': num_puestos,
        'historial_energia': historial_energia_promedio,
        'historial_cola': historial_agentes_en_cola,
        'historial_tiempo': historial_tiempo,
        'agentes': agentes,
        'energia_promedio': np.mean(historial_energia_promedio),
        'energia_inicial': np.mean(historial_energia_promedio[:20]),
        'energia_final': np.mean(historial_energia_promedio[-20:]),
        'saturacion': (sum(1 for n in historial_agentes_en_cola if n >= num_puestos) / len(historial_agentes_en_cola)) * 100,
        'total_recargas': sum(a.num_recargas for a in agentes)
    }


In [50]:

# Ejecutar ambos escenarios
print("\nEjecutando escenario con 5 puestos...")
resultado_5 = ejecutar_experimento(5)
print("Listo")

print("Ejecutando escenario con 1 puesto...")
resultado_1 = ejecutar_experimento(1)
print("Listo\n")



Ejecutando escenario con 5 puestos...
Listo
Ejecutando escenario con 1 puesto...
Listo



### Escenario con capacidad adecuada (5 puestos)

In [51]:
print ("Escenario con capacidad adecuada (5 puestos)")
print(f"Saturación DES: {resultado_5['saturacion']:.1f}%")
print(f"Energía promedio MBA+DS: {resultado_5['energia_promedio']:.2f}")
print(f"Cambio energético: {resultado_5['energia_final'] - resultado_5['energia_inicial']:+.2f}")

Escenario con capacidad adecuada (5 puestos)
Saturación DES: 0.5%
Energía promedio MBA+DS: 6.24
Cambio energético: -1.65


### Escenario con cuello de boterra (1 puestos)

In [52]:
print("Escenario con cuello de boterra (1 puestos)")
print(f"Saturación DES: {resultado_1['saturacion']:.1f}%")
print(f"Energía promedio MBA+DS: {resultado_1['energia_promedio']:.2f}")
print(f"Cambio energético: {resultado_1['energia_final'] - resultado_1['energia_inicial']:+.2f}")

Escenario con cuello de boterra (1 puestos)
Saturación DES: 91.0%
Energía promedio MBA+DS: 4.85
Cambio energético: -3.95


In [55]:
diferencia_energia = resultado_1['energia_promedio'] - resultado_5['energia_promedio']
diferencia_saturacion = resultado_1['saturacion'] - resultado_5['saturacion']
print(f"   • Diferencia en saturación DES: +{diferencia_saturacion:.1f} puntos porcentuales")
print(f"   • Diferencia en energía MBA+DS: {diferencia_energia:.2f} unidades ({(diferencia_energia/resultado_5['energia_promedio'])*100:.1f}%)")
print(f"   • Deterioro temporal (1 puesto): {resultado_1['energia_final'] - resultado_1['energia_inicial']:.2f} unidades")

   • Diferencia en saturación DES: +90.5 puntos porcentuales
   • Diferencia en energía MBA+DS: -1.39 unidades (-22.3%)
   • Deterioro temporal (1 puesto): -3.95 unidades
